In [1]:
import logging; logging.basicConfig(level=logging.INFO)
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import logictensornetworks as ltn
import pandas as pd

In [2]:
df = pd.read_csv("/Users/eduard.hogea/Documents/Facultate/Internship/explorare/KNN99---LTN/datesets/Cantilever/training.csv", on_bad_lines='skip', skiprows=1)
df = df.sample(frac=1) #shuffle
#df.sample(5)
#df.shape
df = df.head(int(len(df)*(70/100)))
df.shape

(25601, 11)

In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
#df = scaler.fit_transform(df)

In [4]:
df.drop_duplicates(keep= 'first', inplace=True) #eliminarea dublurilor liniilor
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna()
print('Null values in dataset are',len(df[df.isnull().any(1)]))
df.shape
from sklearn.utils import shuffle
df = shuffle(df)
df = df.sample(frac=1).reset_index(drop=True)

Null values in dataset are 0


# Data normalization

In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
# selecting numeric attributes columns from data
numeric_col = df.select_dtypes(include='number').columns

In [6]:
# using standard scaler for normalizing
std_scaler = StandardScaler()
def normalization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

In [7]:
df.head(10)

,Pozitie,Severitate def,Severitate inc.,Mod 1,Mod 2,Mod 3,Mod 4,Mod 5,Mod 6,Mod 7,Mod 8
0,0.578,0.000867,0.001593,0.001649,0.002021,0.001754,0.001776,0.001937,0.001615,0.002026,0.001623
1,0.036,0.002141,0.000000,0.001934,0.001468,0.001103,0.000784,0.000522,0.000315,0.000162,0.000062
2,0.212,0.026224,0.001020,0.014208,0.001029,0.006305,0.012387,0.008718,0.001792,0.003125,0.011199
3,0.488,0.001191,0.000567,0.000716,0.001163,0.000570,0.001153,0.000584,0.001137,0.000602,0.001116
4,0.578,0.005124,0.000000,0.000327,0.002529,0.000947,0.001078,0.002031,0.000126,0.002561,0.000178
5,0.954,0.007105,0.000413,0.000413,0.000416,0.000436,0.000493,0.000608,0.000797,0.001069,0.001423
6,0.986,0.009517,0.001593,0.001593,0.001593,0.001594,0.001595,0.001597,0.001600,0.001606,0.001616
7,0.816,0.012434,0.000413,0.000450,0.001280,0.004197,0.007118,0.006914,0.003541,0.000626,0.001348
8,0.374,0.051453,0.001593,0.014161,0.016153,0.017768,0.002573,0.026382,0.009979,0.009043,0.026554
9,0.508,0.003346,0.001020,0.001384,0.002739,0.001033,0.002678,0.001040,0.002661,0.001064,0.002634


In [8]:
# calling the normalization() function
df = normalization(df.copy(),numeric_col)

In [9]:
df.head(10)

,Pozitie,Severitate def,Severitate inc.,Mod 1,Mod 2,Mod 3,Mod 4,Mod 5,Mod 6,Mod 7,Mod 8
0,0.276205,-0.947347,1.635223,-0.420043,-0.453630,-0.505859,-0.515539,-0.499829,-0.546735,-0.493955,-0.552801
1,-1.600274,-0.882543,-1.327174,-0.389760,-0.523423,-0.591816,-0.648523,-0.692326,-0.726775,-0.753027,-0.769843
2,-0.990938,0.342064,0.568372,0.912737,-0.578723,0.095429,0.907387,0.422534,-0.522180,-0.341337,0.777836
3,-0.035387,-0.930842,-0.272588,-0.519007,-0.561863,-0.662222,-0.598975,-0.683777,-0.612822,-0.691877,-0.623266
4,0.276205,-0.730864,-1.327174,-0.560259,-0.389597,-0.612361,-0.609103,-0.486991,-0.752937,-0.419620,-0.753614
5,1.577970,-0.630137,-0.559968,-0.551200,-0.656044,-0.679889,-0.687525,-0.680606,-0.660024,-0.627061,-0.580692
6,1.688759,-0.507483,1.635223,-0.425918,-0.507557,-0.526978,-0.539821,-0.546119,-0.548729,-0.552344,-0.553893
7,1.100195,-0.359133,-0.559968,-0.547292,-0.547058,-0.183031,0.200819,0.177128,-0.279875,-0.688546,-0.591076
8,-0.430071,1.624881,1.635223,0.907774,1.328618,1.609528,-0.408601,2.825053,0.611874,0.481142,2.911629
9,0.033856,-0.821271,0.568372,-0.448082,-0.363122,-0.601001,-0.394540,-0.621762,-0.401820,-0.627737,-0.412411


# --------

In [10]:
x = df[['Severitate def', 'Severitate inc.','Mod 1', 'Mod 2','Mod 3','Mod 4','Mod 5','Mod 6','Mod 7','Mod 8']]
y = df[['Pozitie']]

batch_size= 32
ds_train = tf.data.Dataset.from_tensor_slices((x[:21000],y[:21000])).batch(batch_size)
ds_test = tf.data.Dataset.from_tensor_slices((x[21000:],y[21000:])).batch(batch_size)

Metal device set to: Apple M1


2022-05-17 19:12:02.828154: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-17 19:12:02.828243: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
#df['Freq 1    f1 [Hz]'] = df['Freq 1    f1 [Hz]'].astype(float, errors = 'raise')
df.dtypes

Pozitie            float64
Severitate def     float64
Severitate inc.    float64
Mod 1              float64
Mod 2              float64
Mod 3              float64
Mod 4              float64
Mod 5              float64
Mod 6              float64
Mod 7              float64
Mod 8              float64
dtype: object

In [12]:
f = ltn.Function.MLP(input_shapes=[10],output_shape=[1],hidden_layer_sizes=(100,100,50))

In [13]:
eq = ltn.Predicate.Lambda(
    #lambda args: tf.exp(-0.05*tf.sqrt(tf.reduce_sum(tf.square(args[0]-args[1]),axis=1)))        
    lambda args: 1/(1+0.5*tf.sqrt(tf.reduce_sum(tf.square(args[0]-args[1]),axis=1)))
)

In [14]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")
Exists = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMean(p=2),semantics="exists")

In [15]:
@tf.function
def axioms(x_data, y_data):
    x = ltn.Variable("x", x_data)
    y = ltn.Variable("y", y_data)
    return Forall(ltn.diag(x,y), eq([f(x),y])).tensor

In [16]:
for x, y in ds_test:
    print("Initial sat level %.5f"%axioms(x,y))
    break

Initial sat level 0.66870


2022-05-17 19:12:03.517579: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-17 19:12:03.517631: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [17]:
metrics_dict = {
    'train_sat': tf.keras.metrics.Mean(name='train_sat'),
    'test_sat': tf.keras.metrics.Mean(name='test_sat'),
    'train_accuracy': tf.keras.metrics.RootMeanSquaredError(name="train_accuracy"),
    'test_accuracy': tf.keras.metrics.RootMeanSquaredError(name="test_accuracy")
}

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
@tf.function
def train_step(x, y):
    # sat and update
    with tf.GradientTape() as tape:
        sat = axioms(x, y)
        loss = 1.-sat
    gradients = tape.gradient(loss, f.trainable_variables)
    optimizer.apply_gradients(zip(gradients, f.trainable_variables))
    sat = axioms(x, y)
    metrics_dict['train_sat'](sat)
    # accuracy
    metrics_dict['train_accuracy'](y,f.model(x))
    
@tf.function
def test_step(x, y):
    # sat
    sat = axioms(x, y)
    metrics_dict['test_sat'](sat)
    # accuracy
    metrics_dict['test_accuracy'](y,f.model(x))

In [19]:
from collections import defaultdict

def train(
        epochs,
        metrics_dict, 
        ds_train, 
        ds_test, 
        train_step, 
        test_step,
        track_metrics=1,
        csv_path=None,
        scheduled_parameters=defaultdict(lambda : {})
    ):
    """
    Args:
        epochs: int, number of training epochs.
        metrics_dict: dict, {"metrics_label": tf.keras.metrics instance}.
        ds_train: iterable dataset, e.g. using tf.data.Dataset.
        ds_test: iterable dataset, e.g. using tf.data.Dataset.
        train_step: callable function. the arguments passed to the function
            are the itered elements of ds_train.
        test_step: callable function. the arguments passed to the function
            are the itered elements of ds_test.
        csv_path: (optional) path to create a csv file, to save the metrics.
        scheduled_parameters: (optional) a dictionary that returns kwargs for
            the train_step and test_step functions, for each epoch.
            Call using scheduled_parameters[epoch].
    """
    template = "Epoch {}"
    for metrics_label in metrics_dict.keys():
        template += ", %s: {:.4f}" % metrics_label
    if csv_path is not None:
        csv_file = open(csv_path,"w+")
        headers = ",".join(["Epoch"]+list(metrics_dict.keys()))
        csv_template = ",".join(["{}" for _ in range(len(metrics_dict)+1)])
        csv_file.write(headers+"\n")
    
    for epoch in range(epochs):
        for metrics in metrics_dict.values():
            metrics.reset_states()

        for batch_elements in ds_train:
            train_step(*batch_elements,**scheduled_parameters[epoch])
        for batch_elements in ds_test:
            test_step(*batch_elements,**scheduled_parameters[epoch])

        metrics_results = [metrics.result() for metrics in metrics_dict.values()]
        if epoch%track_metrics == 0:
            print(template.format(epoch,*metrics_results))
        if csv_path is not None:
            csv_file.write(csv_template.format(epoch,*metrics_results)+"\n")
            csv_file.flush()
    if csv_path is not None:
        csv_file.close()

In [ ]:
EPOCHS = 50

modelul = train(
    EPOCHS,
    metrics_dict,
    ds_train,
    ds_test,
    train_step,
    test_step,
    csv_path="regression_results.csv",
    track_metrics=1
)

2022-05-17 19:12:03.706641: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-05-17 19:12:10.626163: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-05-17 19:12:10.799623: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-05-17 19:12:11.337377: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 0, train_sat: 0.8712, test_sat: 0.8813, train_accuracy: 0.4015, test_accuracy: 0.3398
Epoch 1, train_sat: 0.9096, test_sat: 0.9256, train_accuracy: 0.2432, test_accuracy: 0.1920
Epoch 2, train_sat: 0.9252, test_sat: 0.9195, train_accuracy: 0.2004, test_accuracy: 0.2015
Epoch 3, train_sat: 0.9263, test_sat: 0.9136, train_accuracy: 0.1923, test_accuracy: 0.2292
Epoch 4, train_sat: 0.9303, test_sat: 0.9442, train_accuracy: 0.1818, test_accuracy: 0.1291
Epoch 5, train_sat: 0.9415, test_sat: 0.9364, train_accuracy: 0.1519, test_accuracy: 0.1607
Epoch 6, train_sat: 0.9433, test_sat: 0.9388, train_accuracy: 0.1401, test_accuracy: 0.1463
Epoch 7, train_sat: 0.9482, test_sat: 0.9694, train_accuracy: 0.1292, test_accuracy: 0.0674
Epoch 8, train_sat: 0.9482, test_sat: 0.9615, train_accuracy: 0.1266, test_accuracy: 0.0827
Epoch 9, train_sat: 0.9514, test_sat: 0.9140, train_accuracy: 0.1173, test_accuracy: 0.2003
Epoch 10, train_sat: 0.9495, test_sat: 0.9388, train_accuracy: 0.1279, test_accu

In [ ]:
import seaborn as sns
sns.set(font_scale=1.2)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(12,6))
ax = axs[0]
data = {'y_pred':[],'y_true':[]}
for x,y in ds_train:
    data['y_pred'].extend(f.model(x)[:,0].numpy())
    data['y_true'].extend(y[:,0].numpy())
df = pd.DataFrame(data,columns=['y_pred','y_true'])
ax.set_xlim(0,80)
ax.set_ylim(0,80)
sns.regplot("y_true","y_pred",data=df,ax=ax,truncate=False,
            scatter_kws={'alpha':0.5},line_kws={'color':'r','alpha':0.3,'linestyle':'--','linewidth':2})
ax.plot([0,100],[0,100],color="gray",linestyle="--")
ax.set_xlabel("Actual y values")
ax.set_ylabel("Fitted y values")
ax.set_title("Training Data")

ax = axs[1]
data = {'y_pred':[],'y_true':[]}
for x,y in ds_test:
    data['y_pred'].extend(f.model(x)[:,0].numpy())
    data['y_true'].extend(y[:,0].numpy())
df = pd.DataFrame(data,columns=['y_pred','y_true'])
ax.set_ylim(0,80)
ax.set_xlim(0,80)
sns.regplot("y_true","y_pred",data=df,ax=ax,truncate=False,
            scatter_kws={'alpha':0.5},line_kws={'color':'r','alpha':0.3,'linestyle':'--','linewidth':2})
ax.plot([0,100],[0,100],color="gray",linestyle="--")
ax.set_xlabel("Actual y values")
ax.set_ylabel("Fitted y values")
ax.set_title("Test Data")
plt.tight_layout()
plt.savefig("LTN_regresie_1.pdf")

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(df['y_true'], df['y_pred'], c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(df['y_pred']), max(df['y_true']))
p2 = min(min(df['y_pred']), min(df['y_true']))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.savefig("LTN_regresie_2")
plt.show()

In [ ]:
figure, axis = plt.subplots(figsize=(12,8))
x_ax = range(50)
axis.plot(x_ax, df['y_true'].head(50), label="Pozitie", color='c')
axis.plot(x_ax, df['y_pred'].head(50), label="Pozitie-pred", color='b')
axis.legend()


plt.savefig('test.pdf')  
plt.show()

In [ ]:
df